<a href="https://colab.research.google.com/github/thnavear/Qualcomm-DL-Hackathon/blob/master/genai_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 9.9 MB/s eta 0:00:00


In [ ]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_ID", "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following link.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change
# 3.Return your response as a JSON object with the following structure:
#    {
#        "Department": "...",
#        "Priority": "...",
#        "Language": "...",
#        "Type": "..."
#    }
# 4.Convert the JSON object into output.csv with columns "ticket_ID","department","type","priority","language"

import pandas as pd
import json
from groq import Groq
import time
#import backoff

# Install necessary libraries
#!pip install groq

# Initialize the Groq client
client = Groq(api_key="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")

try:
    df = pd.read_csv("/content/test1.csv") # Correct file name
except FileNotFoundError:
    print("Error: test.csv not found. Please upload the file to /content/")
    exit()


# 1. Load training data
try:
    train_df = pd.read_csv("/content/train.csv")
except FileNotFoundError:
    print("Error: train.csv not found. Please upload the file to /content/")
    exit()

'''
batch_size = 10
for i in range(0, len(df), batch_size):
    batch_results = []  # Store results for the current batch

    for j in range(i, min(i + batch_size, len(df))):
        row = df.iloc[j]  # Get the current row
        ticket_id = row['ticket_ID']
        subject = row['ticket_subject']
        body = row['ticket_body']
        #print(ticket_id)
        print(subject)
'''



"\nbatch_size = 10\nfor i in range(0, len(df), batch_size):\n    batch_results = []  # Store results for the current batch\n\n    for j in range(i, min(i + batch_size, len(df))):\n        row = df.iloc[j]  # Get the current row\n        ticket_id = row['ticket_ID']\n        subject = row['ticket_subject']\n        body = row['ticket_body']\n        #print(ticket_id)\n        print(subject)\n"

In [ ]:
# 2. Prepare few-shot examples
few_shot_examples = []
for index, row in train_df.head(5).iterrows():  # Use the first 5 rows as examples
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']
    department = row['department']  # Assuming 'department' is a column
    type = row['type']
    priority = row['priority']
    language = row['language']


    example = f"""Subject: {subject}
Body: {body}

Department: {department}
Type: {type}
Priority: {priority}
Language: {language}
"""

    few_shot_examples.append(example)

NameError: name 'train_df' is not defined

In [ ]:
    prompt = f"""Classify the following email into the given attributes.

## Email to Classify:
Subject: {subject}
Body: {body}

Attributes:
Department: The responsible department (Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry)
Priority(Indicates the urgency and importance of the issue): low, medium, or high
Language(Indicates the language code in which the email is written): The language code (en, de, etc.).
Type: Incident, Request, Problem, or Change

### Rules to Classify IT Tickets into Departments:
- **Technical Support**:
  - Hardware/software malfunctions
  - Connectivity problems
  - System errors/crashes
- **Customer Service**:
  - General inquiries about services/products
  - Non-technical assistance
  - Feedback/complaints about service quality
- **Billing and Payments**:
  - Invoice/billing questions
  - Payment processing/refunds issues
  - Payment plans/billing adjustments
- **Product Support**:
  - Product features/usage inquiries
  - Product-specific troubleshooting
  - Product updates/upgrades requests
- **IT Support**:
  - Internal IT issues
  - Network/server maintenance
  - User account management/access issues
- **Returns and Exchanges**:
  - Product return/exchange requests
  - Warranty/guarantee issues
  - Return policies/procedures inquiries
- **Sales and Pre-Sales**:
  - Product pricing/availability questions
  - Product demonstrations/trials requests
  - Bulk purchases/special offers inquiries
- **Human Resources**:
  - Employee-related inquiries/issues
  - HR policies/procedures requests
  - Payroll/benefits issues
- **Service Outages and Maintenance**:
  - Service outages/disruptions reports
  - Scheduled maintenance notifications
  - Ongoing maintenance updates requests
- **General Inquiry**:
  - Inquiries not fitting above categories
  - Non-specific information/assistance requests

### Rules to Classify IT Tickets into Types:
- **Incident**:
  - Unplanned IT service interruptions/reductions
  - Examples: system crashes, application errors, network outages, hardware failures
  - Goal: restore normal service operation quickly
- **Request**:
  - Formal requests for new/additional items
  - Examples: new hardware/software, system access, information
  - Routine, not indicating existing service failure
- **Problem**:
  - Underlying cause of incidents
  - Examples: recurring system crashes, frequent network outages, persistent performance issues
  - Focus: identify/resolve root cause to prevent future incidents
- **Change**:
  - Modifications to IT services/components
  - Examples: software updates, hardware upgrades, configuration changes, new feature implementations
  - Planned, structured process to minimize risk/impact

### Rules to Classify IT Tickets into Priority Levels:
- **Low Priority**:
  - Minimal impact on operations, address during regular maintenance
  - Examples: minor software bugs, cosmetic issues, non-essential feature requests
  - No immediate action required
- **Medium Priority**:
  - Moderate impact on operations, does not halt critical functions
  - Examples: performance degradation, intermittent connectivity issues, important but non-urgent changes
  - Timely address, no immediate attention required
- **High Priority**:
  - Significant impact on operations, requires immediate attention
  - Examples: system outages, critical application failures, security breaches
  - Resolve quickly to restore normal operations, minimize downtime


Return the output in CSV format with the attributes: "ticket_ID", "department", "type", "priority", "language".
Pls return only the attributes and no need of headers.
Pls DO NOT explain and give the reasoning behind the reponse."""


In [ ]:
results = []
for index, row in df.iterrows():
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']

completion = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_completion_tokens=200,
        top_p=1,
        stream=False
    )
    #print(prompt)
response = completion.choices[0].message.content
    #batch_results.append(response)
    #print(index)
    #print(response)
    #results.extend(batch_results)
    #time.sleep(2.5)


results.append(response)
print(results)

with open("output.csv", "w") as f:
  f.write("ticket_ID,department,type,priority,language\n")  # Write header row
  for result in results:
      f.write(result)

['1,IT Support,Incident,Medium,it']


In [ ]:
!pip install deep_translator
import pandas as pd
from deep_translator import GoogleTranslator

# Load the CSV file
df = pd.read_csv('/content/train.csv')

# Translate each cell in the dataframe
translator = GoogleTranslator(source='auto', target='en')
df_translated = df.applymap(lambda x: translator.translate(x) if isinstance(x, str) else x)

# Save the translated dataframe to a new CSV file
df_translated.to_csv('/content/test_translatedenglish_train.csv', index=False)

print("The translation is complete and saved to 'test_translatedenglish.csv'.")

<ipython-input-3-cda01ceb8c9c>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_translated = df.applymap(lambda x: translator.translate(x) if isinstance(x, str) else x)


The translation is complete and saved to 'test_translatedenglish.csv'.


In [ ]:
'''
##original prompt
Rules for classifying IT tickets into each Department:

1. **Technical Support**:
   - Issues related to hardware or software malfunctions.
   - Troubleshooting connectivity problems.
   - Assistance with system errors or crashes.

2. **Customer Service**:
   - General inquiries about services or products.
   - Requests for information or assistance not related to technical issues.
   - Feedback or complaints about service quality.

3. **Billing and Payments**:
   - Questions about invoices or billing statements.
   - Issues with payment processing or refunds.
   - Requests for payment plans or billing adjustments.

4. **Product Support**:
   - Inquiries about product features or usage.
   - Troubleshooting product-specific issues.
   - Requests for product updates or upgrades.

5. **IT Support**:
   - Internal IT issues within the organization.
   - Network or server maintenance requests.
   - User account management and access issues.

6. **Returns and Exchanges**:
   - Requests to return or exchange products.
   - Issues with product warranties or guarantees.
   - Inquiries about return policies and procedures.

7. **Sales and Pre-Sales**:
   - Questions about product pricing or availability.
   - Requests for product demonstrations or trials.
   - Inquiries about bulk purchases or special offers.

8. **Human Resources**:
   - Employee-related inquiries or issues.
   - Requests for HR policies or procedures.
   - Issues related to payroll or benefits.

9. **Service Outages and Maintenance**:
   - Reports of service outages or disruptions.
   - Scheduled maintenance notifications.
   - Requests for updates on ongoing maintenance activities.

10. **General Inquiry**:
    - Any inquiries that do not fit into the above categories.
    - Requests for information or assistance that are not specific to a particular department.

Rules for classifying IT tickets into each type:

1. **Incident**:
   - Any unplanned interruption to an IT service or reduction in the quality of an IT service.
   - Examples include system crashes, application errors, network outages, or hardware failures.
   - The primary goal is to restore normal service operation as quickly as possible.

2. **Request**:
   - Any formal request for something new or additional.
   - Examples include requests for new hardware or software, access to a system, or information.
   - These are typically routine and do not indicate a failure or issue with existing services.

3. **Problem**:
   - The underlying cause of one or more incidents.
   - Examples include recurring system crashes, frequent network outages, or persistent performance issues.
   - The focus is on identifying and resolving the root cause to prevent future incidents.

4. **Change**:
   - Any modification to an IT service or its components.
   - Examples include software updates, hardware upgrades, configuration changes, or new feature implementations.
   - Changes are typically planned and follow a structured process to minimize risk and impact.

Rules for classifying IT tickets into different priority levels:

1. **Low Priority**:
   - Issues that have minimal impact on operations and can be addressed during regular maintenance.
   - Examples include minor software bugs, cosmetic issues, or requests for non-essential features.
   - No immediate action is required, and the issue can be scheduled for resolution at a convenient time.

2. **Medium Priority**:
   - Issues that have a moderate impact on operations but do not halt critical functions.
   - Examples include performance degradation, intermittent connectivity issues, or requests for important but non-urgent changes.
   - These issues should be addressed in a timely manner but do not require immediate attention.

3. **High Priority**:
   - Issues that have a significant impact on operations and require immediate attention.
   - Examples include system outages, critical application failures, or security breaches.
   - These issues need to be resolved as quickly as possible to restore normal operations and minimize downtime.
'''

In [ ]:
# 1. Load training data
try:
    train_df = pd.read_csv("/content/train.csv")
except FileNotFoundError:
    print("Error: train.csv not found. Please upload the file to /content/")
    exit()

# 2. Prepare few-shot examples
few_shot_examples = []
for index, row in train_df.head(5).iterrows():  # Use the first 5 rows as examples
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']
    department = row['department']  # Assuming 'department' is a column
    type = row['type']
    priority = row['priority']
    language = row['language']


    example = f"""Subject: {subject}
Body: {body}

Department: {department}
Type: {type}
Priority: {priority}
Language: {language}
"""

    few_shot_examples.append(example)
    print(few_shot_examples)


In [ ]:
output_df = pd.DataFrame(processed_results)
output_df.to_csv("output.csv", index=False)

In [ ]:
print(few_shot_examples)
#print(processed_results)

['Subject: Discrepancia de facturación en Google Workspace\nBody: Estimado equipo de soporte de TI,\n\nEstoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.\n\nSaludos,\n\n<name>\n\nDepartment: Billing and Payments\nType: Incident\nPriority: low\nLanguage: es\n', 'Subject: Urgent Consultation Request for Critical IT Issues\nBody: Dear IT Services Support Team, I hope this message finds you well. My name is <name> and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please contact us at your earliest conve

In [2]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_ID", "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following link.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change
# 3.Return your response as a JSON object with the following structure:
#    {
#        "Department": "...",
#        "Priority": "...",
#        "Language": "...",
#        "Type": "..."
#    }
# 4.Convert the JSON object into output.csv with columns "ticket_ID","department","type","priority","language"

# Install necessary libraries
!pip install groq
import pandas as pd
import json
from groq import Groq
import time
#import backoff

# Initialize the Groq client
client = Groq(api_key="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")

try:
    df = pd.read_csv("/content/test.csv") # Correct file name
except FileNotFoundError:
    print("Error: test.csv not found. Please upload the file to /content/")
    exit()


# 1. Load training data
try:
    train_df = pd.read_csv("/content/train.csv")
except FileNotFoundError:
    print("Error: train.csv not found. Please upload the file to /content/")
    exit()

# 2. Prepare few-shot examples
few_shot_examples = []
for index, row in train_df.head(10).iterrows():  # Use the first 10 rows as examples
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']
    department = row['department']  # Assuming 'department' is a column
    type = row['type']
    priority = row['priority']
    language = row['language']


    example = f"""Subject: {subject}
Body: {body}

Department: {department}
Type: {type}
Priority: {priority}
Language: {language}
"""

#format instructions
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
format_instructions


results = []
#for index, row in df.iloc[344:].iterrows():
for index, row in df.iterrows():
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']

    prompt = f"""Classify the following email into the given attributes.
## Email to Classify:
ticket_id: {ticket_id}
Subject: {subject}
Body: {body}

Output format instructions:
             {format_instructions}

Attributes:
Department: The responsible department (Technical Support, Customer Service, Billing and Payments, Product Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry)
Priority(Indicates the urgency and importance of the issue): low, medium, or high
Language(Indicates the language code in which the email is written): The language code (en, de, etc.).
Type: Incident, Request, Problem, or Change


### Rules to Classify IT Tickets into Departments:
1.Understand the User's Issue: Carefully read the ticket {subject} and {body} to grasp the core problem or request.
2. Look for corresponding keywords(below) in {subject} and {body} and map department accordingly
3. Consider the Context: Sometimes, the words may be ambiguous. Use the surrounding information to make the best choice.
4. Use the below rules as a guide
- **Billing and Payments**:
  - Invoice/billing questions,Payment processing/refunds issues,Payment plans/billing adjustments
  - Keywords: "billing", "invoice", "payment", "charge", "refund"
- **Product Support**:
  - Product features/usage inquiries,Product-specific troubleshooting,Product updates/upgrades requests
  - Keywords: "activation", "product", "software", "installation"
- **Returns and Exchanges**:
  - Return/exchange requests,Warranty/guarantee issues,Return policies/procedures inquiries
  - Keywords: "defective", "exchange", "return", "replacement"
- **Sales and Pre-Sales**:
  - Product pricing/availability questions,Product demonstrations/trials requests,Bulk purchases/special offers inquiries
  - Keywords: "purchase", "buy", "quote", "pricing", "sales"
- **Human Resources**:
  - Employee-related inquiries/issues,HR policies/procedures requests,Payroll/benefits issues
  - Keywords: "employee", "HR", "records", "management"
- **Service Outages and Maintenance**:
  - Service outages/disruptions reports,Scheduled maintenance notifications,Ongoing maintenance updates requests
  - Keywords: "outage", "downtime", "maintenance", "service disruption"
- **Technical Support**:
  - Hardware/software malfunctions,Connectivity problems
- **Customer Service**:
  - Inquiries about services/products,Non-technical assistance,Feedback/complaints about service quality
  - Keywords: "Question," "inquiry," "help," "information," "feedback," "complaint," "service," "product," "general," "how to," "assistance."
- **General Inquiry**:
  - Inquiries not fitting above categories,Non-specific information/assistance requests
  - Keywords: "General," "other," "not sure," "unclear," "information," "help,"


### Rules to Classify IT Tickets into Types:
Initial Check:
- Service Down? Is something broken or degraded?
If yes, consider Incident/Problem.
If no, consider Request/Change.
Incident Questions:
-  Unexpected? Was the disruption unplanned?
-  Restore Quickly? Is the priority to get it working ASAP?
-  Workaround? Can users work, even partially, while fixing it?
If YES, it is likely an Incident.
Request Questions:
-  New Item? Is the user asking for something new?
-  Standard Item? Is this something we usually provide?
-  Routine? Is this a typical request?
If YES, it is likely an Request.
Change Questions:
-  Modification? Does it alter the IT environment?
-  Planned? Was this change scheduled?
-  Upgrade/New Features? Does it add or improve IT services?
If YES, it is likely an Change.
Problem Questions:
-  Recurring? Has this happened before?
-  Multiple Incidents? Is it the root of several issues?
-  Needs Investigation? Does the cause need further research?
If YES, it is likely an Problem.

### Rules to Classify IT Tickets into Priority Levels:
1.high Priority
-  Is a critical system down?
-  Can users do their core jobs?
-  Is there a security breach?
-  Is there a financial/legal risk?
-  Are many users affected?
-  Is a workaround needed now?
If the answer to any of these questions is "yes", it is likely a high priority ticket
2.medium Priority
-  Is a core system slow/intermittent?
-  Is user productivity impacted?
-  Is a business process delayed?
-  Are a few users affected?
-  Is there some time before a solution is needed?
If the answer to any of these questions is "yes" and it is not a high priority, then it is likely a medium priority ticket.
3.low Priority
-  Is it a minor issue?
-  Is it a non-essential request?
-  Is there minimal impact?
-  Can it wait for maintenance?
-  Is it a general question?
If the answer to any of these questions is "yes", it is likely a low priority ticket.

#Return the output in CSV format with the attributes: "ticket_ID", "department", "type", "priority", "language".
Pls return only the attributes (case-sensitive) and no need of headers.
Pls DO NOT explain and give the reasoning behind the reponse."""

    completion = client.chat.completions.create(
        model="gemma2-9b-it",
        model="llama3-70b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_completion_tokens=200,
        top_p=1,
        stream=False
    )
    time.sleep(2.5)

    response = completion.choices[0].message.content
   # print(index)
    print(response)

    results.append(response)

    with open("output.csv", "w") as f:
      f.write("ticket_ID,department,type,priority,language\n")  # Write header row
      for result in results:
          f.write(result)


1,Technical Support,Incident,high,it
2,Billing and Payments,Request,low,de
3,Technical Support,Problem,medium,fr
4,Billing and Payments,Request,medium,it
5,Technical Support,Incident,high,it
6,Technical Support,Incident,high,it
7,Technical Support,Incident,high,it
8,Technical Support,Incident,high,pt
9,Technical Support,Request,medium,es
10,Technical Support,Problem,high,it
11,Technical Support,Problem,high,it
12,Technical Support,Problem,high,it
13,Billing and Payments,Problem,high,it
14, Technical Support, Change, high, de
15,Technical Support,Incident,high,de
16,Technical Support,Problem,high,it
17,Technical Support,Incident,high,pt
18,Technical Support,Problem,high,de
19, Technical Support, Problem, high, it
20,Technical Support,Problem,high,it
21,Sales and Pre-Sales,Request,low,pt
22,Technical Support,Problem,high,de
23,Technical Support,Problem,high,pt
24,Sales and Pre-Sales,Request,low,fr
25,Technical Support,Problem,high,it
26,Technical Support,Problem,high,it
27,Technical Supp

In [5]:
!pip install langdetect
import pandas as pd
from langdetect import detect

# Load the CSV file
try:
    df = pd.read_csv('/content/test.csv')
except FileNotFoundError:
    print("Error: train.csv not found. Please upload the file to /content/")
    exit()

# Function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'  # Return 'unknown' for any errors

# Concatenate 'ticket_subject' and 'ticket_body'
df['combined_text'] = df['ticket_subject'].fillna('') + ' ' + df['ticket_body'].fillna('')

 #Apply language detection to the combined text
df['combined_language'] = df['combined_text'].apply(lambda x: detect_language(x) if isinstance(x, str) else 'unknown')

# Apply language detection to 'ticket_subject' and 'ticket_body'
#df['subject_language'] = df['ticket_subject'].apply(lambda x: detect_language(x) if isinstance(x, str) else 'unknown')
#df['body_language'] = df['ticket_body'].apply(lambda x: detect_language(x) if isinstance(x, str) else 'unknown')

# Save the DataFrame with the new language columns
df.to_csv('/content/train_with_language.csv', index=False)

print("The language detection is complete and saved to 'train_with_language.csv'.")

The language detection is complete and saved to 'train_with_language.csv'.


In [ ]:
#!pip install langchain_groq

#!pip install langchain
from langchain import PromptTemplate

import os
from langchain_groq import ChatGroq

prompt_template = PromptTemplate(
    input_variables=[],  # No input variables needed in this case
    template=prompt  # Your existing prompt string
)

# Set the Groq API key as an environment variable
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key

# Initialize the ChatGroq instance
chatgroq = ChatGroq(model="gemma2-9b-it", temperature=0)

# create a simple LLM Chain - more on this later
llm_chain = (prompt_template
              |
            chatgroq
              |
            output_parser)

# run the chain
response = llm_chain.invoke({'format_instructions': format_instructions})
response

['1', 'Technical Support', 'Incident', 'High', 'it']

In [ ]:
# prompt: 1. write a code to import /content/test.csv
# 2.use model "llama-3.3-70b-versatile" from groq to remove all stop words and find all unique words along with their number of occurences

import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

# Load the dataframe
try:
    df = pd.read_csv('/content/test_translatedenglish_full.csv')
except FileNotFoundError:
    print("Error: test.csv not found. Please upload the file to /content/")
    exit()

# Download stopwords if not already present
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
'''
# Function to remove stop words and count unique words
def process_text(text):
    if isinstance(text, str):  # Check if the input is a string
        words = text.lower().split()
        words = [word for word in words if word not in stop_words and word.isalnum()]
        word_counts = Counter(words)
        return word_counts
    else:
        return Counter()  # Return an empty counter for non-string values
'''

# Combine text columns and process them
all_words = Counter()

for col in ['ticket_subject', 'ticket_body']:
  if col in df.columns:  #check if the columns exists in the dataframe
      all_words.update(sum(df[col].apply(process_text).tolist(),Counter()))
  else:
      print(f"Warning: Column '{col}' not found in the dataframe.")

# Print unique words and their counts
for word, count in all_words.items():
    print(f"{word}: {count}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


problem: 149
zoom: 42
screen: 52
sharing: 4
feature: 10
billing: 70
discrepancies: 17
google: 23
workspace: 12
problems: 51
wireless: 62
function: 4
epson: 17
ecotank: 17
need: 135
help: 212
chrome: 11
crashes: 22
urgent: 125
consultation: 12
request: 266
critical: 92
issues: 206
installation: 16
failure: 17
windows: 21
11: 10
pro: 41
assistance: 197
server: 83
administration: 13
slow: 14
network: 95
connectivity: 85
issue: 491
reported: 18
cisco: 71
router: 76
isr4331: 55
device: 53
hp: 53
deskjet: 48
3755: 41
printer: 90
subscription: 33
correction: 1
change: 35
jira: 65
ticket: 26
setting: 36
laserjet: 3
m15w: 2
aws: 173
cost: 30
optimization: 22
deployment: 20
flickering: 31
microsoft: 49
surface: 35
7: 20
query: 6
macbook: 76
air: 72
battery: 25
overheating: 13
information: 98
touch: 4
functions: 3
immediate: 133
needed: 41
consulting: 88
installations: 1
refund: 12
returned: 1
m1: 54
discrepancy: 14
common: 9
paper: 31
jams: 8
canon: 34
printers: 3
replacement: 38
setup: 31
consu